In [541]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
 
import twitter_credentials
import numpy as np
import pandas as pd
import datetime as dt
import re


# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None,list_id=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_list=list_id
        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_list_tweets(self):
        tweets = []
        for tweet in Cursor(self.twitter_client.list_timeline, list_id=self.twitter_list).pages():
            tweets=tweet+tweets
        return tweets
    
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['handle'] = np.array([tweet.user.screen_name for tweet in tweets])
        df['name'] = np.array([tweet.user.name for tweet in tweets])

        return df

In [524]:
pd.options.display.max_colwidth = 400
pd.set_option('display.max_rows', 100)

In [960]:
popular_artist_list='1255643235386605569'
rkm_artist_list='1090078355246985217'
chill_artist_list='1256130288322842624'

In [961]:
twitter_popular_list = TwitterClient(None,popular_artist_list)
twitter_rkm_list = TwitterClient(None,rkm_artist_list)
twitter_chill_list=TwitterClient(None,chill_artist_list)

tweets_pop = twitter_popular_list.get_list_tweets()
tweets_rkm=twitter_rkm_list.get_list_tweets()
tweets_chilled_cheese=twitter_chill_list.get_list_tweets()

popular_artists=tweet_analyzer.tweets_to_data_frame(tweets_pop)
chill_artists=tweet_analyzer.tweets_to_data_frame(tweets_chilled_cheese)
rkm_artists=tweet_analyzer.tweets_to_data_frame(tweets_rkm)

rkm_tweets=rkm_artists.Tweets
popular_tweets=popular_artists.Tweets
chilled_tweets=chill_artists.Tweets

In [962]:
show_tweets=[]
checks=['new music','is out','out soon','out now','new song','tomorrow','new release',
        'new single','coming soon','listen to','this month','friday'
        ,'more days','drops','next week','free ','download','presave','pre-save','available','days'
        ,'music video','listen on','album','noon','midnight','download'
        ,dt.date.today().strftime("%B")[:3].lower()
        ,(dt.date.today()+pd.DateOffset(weeks=1)).strftime("%B")[:3].lower(),
        '.{}'.format(dt.date.today().strftime("%m")[1:]),
        '/{}'.format(dt.date.today().strftime("%m")[1:]),
        '-{}'.format(dt.date.today().strftime("%m")[1:]),
        '/{}'.format((dt.date.today()+pd.DateOffset(weeks=1)).strftime("%m")[1:]),
        '.{}'.format((dt.date.today()+pd.DateOffset(weeks=1)).strftime("%m")[1:]),
        '-{}'.format((dt.date.today()+pd.DateOffset(weeks=1)).strftime("%m")[1:]),
        '{}'.format(dt.date.today().year)
       ]

for tweet in popular_tweets:
    flag=False
    for check in checks:
        if check in tweet.lower() or bool(re.match("([\s\S]*[0-9]|0[0-9]|1[0-9]|2[0-3])\s*([AaPp][Mm])", tweet.lower())) or bool(re.match(" ([\s\S]*[0-9]|0[0-9]|1[0-9]|2[0-3]):([0-5][0-9])\s*([AaPp][Mm])", tweet.lower())) or bool(re.match("[\s\S]*(([\s\W]mon|[\s\W]tues|[\s\W]wed(nes)?|[\s\W]thur(s)?|[\s\W]fri|[\s\W]sat(ur)?|[\s\W]sun)(day)?[\s\W])",tweet.lower())) :
            show_tweets.append(True)
            flag=True
            break
    if flag==False:
        show_tweets.append(False)

In [963]:
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 200)

popular_artists[show_tweets][['Tweets','name','handle','date']].sort_values('date',ascending=False).reset_index(drop=True)

,Tweets,name,handle,date
0,"RT @Genius: brandy is back today with her new single, “baby mama,” and she brought chance the rapper along\n\nhttps://t.co/Tbz2uIa6HN",b r a n d y,4everBrandy,2020-05-03 19:00:25
1,IF U DONT HAVE #goodtoknow yet DOWNLOAD IT RN so we can listen together in 3 minutesssssssss #goodtoknow,JoJo.,iamjojo,2020-05-03 18:57:09
2,one from quarantine. coast of maine. march 2020. https://t.co/O8DplkadTF,Maggie Rogers,maggierogers,2020-05-03 18:41:48
3,let’s listen to my album together 🧡🙏🏼🥰 we’ll pop off at 3 pm EST/ noon PST send any questions u have here 👉🏼👉🏿👉🏾👉 #goodtoknow,JoJo.,iamjojo,2020-05-03 18:20:06
4,Free tonight? Let’s have a #HappinessContinues watch party here on Twitter 😎 See you at 5pm PT / 8pm ET!… https://t.co/pFCjlQMTmq,Jonas Brothers,jonasbrothers,2020-05-03 18:09:11
5,RT @jojolevesqueus: .@iamjojo is doing a Q&amp;A / Listening Party on Twitter at 12 PM PST / 3 PM EST. That’s in a little over an hour. #Goodto…,JoJo.,iamjojo,2020-05-03 18:02:33
6,#BabyMama is featured on the R&amp;B Now playlist! Listen on @AppleMusic ❤️ https://t.co/fxx9yhEGqe,b r a n d y,4everBrandy,2020-05-03 17:30:01
7,5 days #stuckwithu @ArianaGrande https://t.co/3J5JRRo03X,Justin Bieber,justinbieber,2020-05-03 17:16:22
8,RT @Nickelodeon: and the crowd goes wild! congrats on that #KCA2020 win Lil Nas X 💚 https://t.co/6EimVg9m8M,nope,LilNasX,2020-05-03 17:16:18
9,YOU KNOW WHAT TIME IT IS. A VERY LAZER SUNDAY TODAY AT 3PM PT https://t.co/7zsEaGjlH2,MAJOR LAZER,MAJORLAZER,2020-05-03 17:12:22
